![Retip](../images/retip_logo.png)
# Retip: Retention Time Prediction for Metabolomics

### Identiying False Annotations

Given an annotated metabolomics dataset, we can use retention time prediction to identify likely misannotated features.  To begin, we import Retip and load our data as usual. Outlier analysis requires the same type of input as model building, namely a table of compound Name, SMILES (or other supported chemical identifier) and retention time.

Since we are not building this model to apply on other datasets, we do not need a to split it into separate training and test set.

In [1]:
try:
    import retip
except:
    # add the parent directory to the path to load the Retip library locally in case it isn't installed
    import os, sys
    sys.path.insert(1, os.path.join(sys.path[0], '..'))
    
    import retip

In [3]:
dataset = retip.Dataset().load_retip_dataset('../example_data/tomato_annotations.csv')
dataset.calculate_descriptors()
dataset.preprocess_features('metabolomics')

Calculating descriptors for training dataset


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1164/1164 [05:45<00:00,  3.37it/s]


Reduced feature set from 1613 to 817


In [4]:
dataset.describe()

Training (1164, 821)


In [5]:
trainer = retip.XGBoostTrainer(dataset, cv=5)
trainer.train()

Fitting 5 folds for each of 56 candidates, totalling 280 fits
Training completed in 0:16:28.270350 with best RMSE -10.631


Running the `outlier_identification` function will provide two results:

1. A plot showing the distribution of real vs. predicted retention times overlaid by a simple linear fit with 95% confidence intervals, with any annotations outside of this CI window are highlighted in red
2. A table listing the outliers with their name, retention time and predicted retention time

In [7]:
outliers = retip.visualization.outlier_identification(trainer, dataset, 'RTP', confidence_interval=95)
outliers

Loading BokehJS ...

AttributeError: 'Dataset' object has no attribute 'df'

You can also select a different confidence interval threshold, for example 90% CI:

In [ ]:
outliers = retip.outlier_identification(trainer, dataset, confidence_interval=90)
outliers